In [56]:
import os
import sys
import numpy as np
import pandas as pd
from sisepuede.core.support_classes import Regions, TimePeriods
from sisepuede.core.model_attributes import ModelAttributes

from sisepuede.manager.sisepuede_examples import SISEPUEDEExamples
from sisepuede.manager.sisepuede_file_structure import SISEPUEDEFileStructure
import sisepuede.core.support_classes as sc
import sisepuede.transformers as trf
import sisepuede.utilities._plotting as spu
import sisepuede.utilities._toolbox as sf
import sisepuede as si

In [57]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
def get_iea_transport_data(cw_file_path, iea_transport_data_path):

    # Load the IEA transport data
    df_iea_transport = pd.read_csv(iea_transport_data_path, encoding='ISO-8859-1')

    # Load the CW file
    cw_df = pd.read_csv(cw_file_path)

    # Create a dictionary for mapping old country names to new country names
    iea_country_name_mapping = dict(zip(cw_df['old_country_name'], cw_df['new_country_name']))

    # Create a copy of the dataframe to avoid modifying the original
    df_iea_transport_copy = df_iea_transport.copy()

    # Replace the country names in the copied dataframe using the mapping dictionary
    df_iea_transport_copy['COUNTRY'] = df_iea_transport_copy['COUNTRY'].replace(iea_country_name_mapping)

    return df_iea_transport_copy

In [59]:
# Creates historical dataframe skeleton

def create_historical_dataframe_skeleton(df_freight):
    # Get unique ISO alpha-3 country codes from the freight dataframe
    iso_alpha_3_codes = df_freight['LOCATION'].unique()
    
    # Get unique years from the freight dataframe
    years = df_freight['TIME'].unique()
    
    # Create a list of tuples containing all combinations of country codes and years
    iso_years = [(iso, year) for iso in iso_alpha_3_codes for year in years]
    
    # Create a dataframe with columns 'iso_alpha_3' and 'year' from the list of tuples
    df_historical = pd.DataFrame(iso_years, columns=['iso_alpha_3', 'year'])
    
    # # Add a new column for the variable name with initial value set to 0
    # df_historical[var_names] = 0

    df_historical.sort_values(by=['iso_alpha_3', 'year'], ascending=True, inplace=True)
    df_historical.reset_index(drop=True, inplace=True)
    
    return df_historical

In [60]:
def calculate_mean_value_oecd(df_freight):
    """
    Calculate the mean value of OECD data for each subject and year.

    Parameters:
    df_freight (pd.DataFrame): DataFrame containing freight data.

    Returns:
    dict: Dictionary containing mean values for each subject and year.
    """
    # Initialize the dictionary to store mean values
    mean_value_oecd = {}

    # Get unique subjects from the freight dataframe
    subjects = df_freight['SUBJECT'].unique()
    
    # Extract and sort unique years from the freight dataframe
    years = sorted(df_freight['TIME'].unique())

    # Iterate over each subject
    for subject in subjects:
        if subject == 'INLAND':
            continue
        mean_value_oecd[subject] = {}
        # Iterate over each year
        for y in years:
            vect = []
            # Iterate over each unique location
            for c in np.unique(df_freight['LOCATION'].values):
                try:
                    # Get the data value for the specific location, year, and subject
                    data = df_freight.loc[
                        (df_freight['LOCATION'] == c) &
                        (df_freight['TIME'] == y) &
                        (df_freight['SUBJECT'] == subject)
                    ]['Value'].values[0]
                except IndexError:
                    data = 0

                # Get the total value for the specific location and year
                total = df_freight.loc[
                    (df_freight['LOCATION'] == c) &
                    (df_freight['TIME'] == y)
                ]['Value'].sum()

                # Calculate the ratio and append to the vector
                if total > 1:
                    if subject == 'ROAD':
                        inland_data = df_freight.loc[
                            (df_freight['LOCATION'] == c) &
                            (df_freight['TIME'] == y) &
                            (df_freight['SUBJECT'] == 'INLAND')
                        ]['Value'].values[0] if 'INLAND' in subjects else 0
                        vect.append(float(data + inland_data) / total)
                    else:
                        vect.append(float(data) / total)
                else:
                    vect.append(0)

            # Calculate the mean value for the subject and year
            mean_value_oecd[subject][y] = np.mean(vect)

    return mean_value_oecd

In [61]:
def update_historical_data(df_historical_skeleton, df_freight):
    
    df_historical = df_historical_skeleton.copy()
    freight_data_iso_codes = np.unique(df_freight['LOCATION'].values)

    for iso_alpha_3, year in zip(df_historical['iso_alpha_3'].values, df_historical['year'].values):
        if iso_alpha_3 in freight_data_iso_codes:
            subject_data = {}
            total = 0
            for subject in df_freight['SUBJECT'].unique():
                try:
                    subject_data[subject] = df_freight.loc[
                        (df_freight['LOCATION'] == iso_alpha_3) &
                        (df_freight['TIME'] == year) &
                        (df_freight['SUBJECT'] == subject)
                    ]['Value'].values[0]
                except IndexError:
                    subject_data[subject] = 0
                total += subject_data[subject]

            for subject in df_freight['SUBJECT'].unique():
                if subject == 'INLAND':
                    continue
                if total > 1:
                    if subject == 'ROAD':
                        frac_value = (subject_data['ROAD'] + subject_data.get('INLAND', 0)) / total
                    else:
                        frac_value = subject_data[subject] / total
                else:
                    frac_value = 0
                df_historical.loc[
                    (df_historical['iso_alpha_3'] == iso_alpha_3) &
                    (df_historical['year'] == year), f'frac_trns_mtkm_dem_freight_{subject.lower()}_freight'
                ] = frac_value
        else:
            for subject in df_freight['SUBJECT'].unique():
                if subject == 'INLAND':
                    continue
                df_historical.loc[
                    (df_historical['iso_alpha_3'] == iso_alpha_3) &
                    (df_historical['year'] == year), f'frac_trns_mtkm_dem_freight_{subject.lower()}_freight'
                ] = np.nan

    return df_historical


In [62]:
def main():

    parent_dir = os.path.abspath('..')
    raw_data_dir_path = os.path.join(parent_dir, 'data', 'raw')
    # cw_dir_path = os.path.join(parent_dir, 'data', 'cw')
    processed_data_dir_path = os.path.join(parent_dir, 'data', 'processed')

    # iso_alpha_3_cw_df = pd.read_csv(os.path.join(cw_dir_path, 'iso_alpha_3_cw.csv'))
    df_freight=pd.read_csv(os.path.join(raw_data_dir_path, 'imputed_freight_data.csv'))

    df_freight = df_freight.fillna(0)
    # print(df_freight)
    
    # df_iea_transport = get_iea_transport_data(os.path.join(cw_dir_path, 'IEA_country_name_cw.csv'), os.path.join(raw_data_dir_path, 'IEA_transport_data.csv')) # This is used for imputation but it does not seem to be used in the code
    
    df_historical_skeleton = create_historical_dataframe_skeleton(df_freight)

    df_historical = update_historical_data(df_historical_skeleton, df_freight)

    # Impute missing values in the historical dataframe frac columns with the mean value in each column
    for col in df_historical.columns:
        if 'frac' in col:
            df_historical[col] = df_historical[col].fillna(df_historical[col].mean())
            
    df_historical.to_csv(os.path.join(processed_data_dir_path, 'historical', f'frac_trns_mtkm.csv'), index=False)
    
    return df_historical

In [63]:
df = main()
df

,iso_alpha_3,year,frac_trns_mtkm_dem_freight_coast_freight,frac_trns_mtkm_dem_freight_rail_freight,frac_trns_mtkm_dem_freight_road_freight
0,ABW,2011,0.107914,0.165468,0.726619
1,ABW,2012,0.107914,0.165468,0.726619
2,ABW,2013,0.240144,0.368221,0.391635
3,ABW,2014,0.240145,0.368223,0.391632
4,ABW,2015,0.240141,0.368216,0.391643
...,...,...,...,...,...
1766,ZWE,2017,0.107914,0.165468,0.726619
1767,ZWE,2018,0.107914,0.165468,0.726619
1768,ZWE,2019,0.107914,0.165468,0.726619
1769,ZWE,2020,0.107914,0.165468,0.726619


In [64]:
df.isnull().sum()

iso_alpha_3                                 0
year                                        0
frac_trns_mtkm_dem_freight_coast_freight    0
frac_trns_mtkm_dem_freight_rail_freight     0
frac_trns_mtkm_dem_freight_road_freight     0
dtype: int64